# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f00e855a340>
├── 'a' --> tensor([[ 0.9657,  0.5533, -0.0730],
│                   [ 0.0205,  0.6028,  1.2843]])
└── 'x' --> <FastTreeValue 0x7f00e855a7f0>
    └── 'c' --> tensor([[ 2.4693, -0.9482,  0.4203, -0.2541],
                        [-0.5634,  1.1323, -0.0617, -0.3299],
                        [-0.4680,  0.0155,  2.0219,  0.3427]])

In [4]:
t.a

tensor([[ 0.9657,  0.5533, -0.0730],
        [ 0.0205,  0.6028,  1.2843]])

In [5]:
%timeit t.a

62.9 ns ± 0.0267 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f00e855a340>
├── 'a' --> tensor([[ 0.3956,  0.7540,  0.1625],
│                   [-0.6691,  1.3619,  0.2150]])
└── 'x' --> <FastTreeValue 0x7f00e855a7f0>
    └── 'c' --> tensor([[ 2.4693, -0.9482,  0.4203, -0.2541],
                        [-0.5634,  1.1323, -0.0617, -0.3299],
                        [-0.4680,  0.0155,  2.0219,  0.3427]])

In [7]:
%timeit t.a = new_value

61.9 ns ± 0.0327 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9657,  0.5533, -0.0730],
               [ 0.0205,  0.6028,  1.2843]]),
    x: Batch(
           c: tensor([[ 2.4693, -0.9482,  0.4203, -0.2541],
                      [-0.5634,  1.1323, -0.0617, -0.3299],
                      [-0.4680,  0.0155,  2.0219,  0.3427]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9657,  0.5533, -0.0730],
        [ 0.0205,  0.6028,  1.2843]])

In [11]:
%timeit b.a

55.7 ns ± 0.0697 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.1155, -0.0203, -0.3145],
               [-0.1606, -0.7114,  1.1686]]),
    x: Batch(
           c: tensor([[ 2.4693, -0.9482,  0.4203, -0.2541],
                      [-0.5634,  1.1323, -0.0617, -0.3299],
                      [-0.4680,  0.0155,  2.0219,  0.3427]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.104 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

956 ns ± 15.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 66.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 716 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f00e855aa30>
├── 'a' --> tensor([[[ 0.9657,  0.5533, -0.0730],
│                    [ 0.0205,  0.6028,  1.2843]],
│           
│                   [[ 0.9657,  0.5533, -0.0730],
│                    [ 0.0205,  0.6028,  1.2843]],
│           
│                   [[ 0.9657,  0.5533, -0.0730],
│                    [ 0.0205,  0.6028,  1.2843]],
│           
│                   [[ 0.9657,  0.5533, -0.0730],
│                    [ 0.0205,  0.6028,  1.2843]],
│           
│                   [[ 0.9657,  0.5533, -0.0730],
│                    [ 0.0205,  0.6028,  1.2843]],
│           
│                   [[ 0.9657,  0.5533, -0.0730],
│                    [ 0.0205,  0.6028,  1.2843]],
│           
│                   [[ 0.9657,  0.5533, -0.0730],
│                    [ 0.0205,  0.6028,  1.2843]],
│           
│                   [[ 0.9657,  0.5533, -0.0730],
│                    [ 0.0205,  0.6028,  1.2843]]])
└── 'x' --> <FastTreeValue 0x7f00e855ab50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 81.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f0031e22d30>
├── 'a' --> tensor([[ 0.9657,  0.5533, -0.0730],
│                   [ 0.0205,  0.6028,  1.2843],
│                   [ 0.9657,  0.5533, -0.0730],
│                   [ 0.0205,  0.6028,  1.2843],
│                   [ 0.9657,  0.5533, -0.0730],
│                   [ 0.0205,  0.6028,  1.2843],
│                   [ 0.9657,  0.5533, -0.0730],
│                   [ 0.0205,  0.6028,  1.2843],
│                   [ 0.9657,  0.5533, -0.0730],
│                   [ 0.0205,  0.6028,  1.2843],
│                   [ 0.9657,  0.5533, -0.0730],
│                   [ 0.0205,  0.6028,  1.2843],
│                   [ 0.9657,  0.5533, -0.0730],
│                   [ 0.0205,  0.6028,  1.2843],
│                   [ 0.9657,  0.5533, -0.0730],
│                   [ 0.0205,  0.6028,  1.2843]])
└── 'x' --> <FastTreeValue 0x7f00e85618b0>
    └── 'c' --> tensor([[ 2.4693, -0.9482,  0.4203, -0.2541],
                        [-0.5634,  1.1323, -0.0617, -0.3299],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 86.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.9 µs ± 84.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 2.4693, -0.9482,  0.4203, -0.2541],
                       [-0.5634,  1.1323, -0.0617, -0.3299],
                       [-0.4680,  0.0155,  2.0219,  0.3427]],
              
                      [[ 2.4693, -0.9482,  0.4203, -0.2541],
                       [-0.5634,  1.1323, -0.0617, -0.3299],
                       [-0.4680,  0.0155,  2.0219,  0.3427]],
              
                      [[ 2.4693, -0.9482,  0.4203, -0.2541],
                       [-0.5634,  1.1323, -0.0617, -0.3299],
                       [-0.4680,  0.0155,  2.0219,  0.3427]],
              
                      [[ 2.4693, -0.9482,  0.4203, -0.2541],
                       [-0.5634,  1.1323, -0.0617, -0.3299],
                       [-0.4680,  0.0155,  2.0219,  0.3427]],
              
                      [[ 2.4693, -0.9482,  0.4203, -0.2541],
                       [-0.5634,  1.1323, -0.0617, -0.3299],
                       [-0.4680,  0.0155,  2.0219,  0.3427]],

In [26]:
%timeit Batch.stack(batches)

78 µs ± 55.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 2.4693, -0.9482,  0.4203, -0.2541],
                      [-0.5634,  1.1323, -0.0617, -0.3299],
                      [-0.4680,  0.0155,  2.0219,  0.3427],
                      [ 2.4693, -0.9482,  0.4203, -0.2541],
                      [-0.5634,  1.1323, -0.0617, -0.3299],
                      [-0.4680,  0.0155,  2.0219,  0.3427],
                      [ 2.4693, -0.9482,  0.4203, -0.2541],
                      [-0.5634,  1.1323, -0.0617, -0.3299],
                      [-0.4680,  0.0155,  2.0219,  0.3427],
                      [ 2.4693, -0.9482,  0.4203, -0.2541],
                      [-0.5634,  1.1323, -0.0617, -0.3299],
                      [-0.4680,  0.0155,  2.0219,  0.3427],
                      [ 2.4693, -0.9482,  0.4203, -0.2541],
                      [-0.5634,  1.1323, -0.0617, -0.3299],
                      [-0.4680,  0.0155,  2.0219,  0.3427],
                      [ 2.4693, -0.9482,  0.4203, -0.2541],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 77.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

302 µs ± 1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
